### CONCERNS
1. list of directions/positions are never emptied during an episode, which makes the simulation slower and slower after each timestep

### IDEAS
other possible features:
1. robot is fallen down or not
2. distance to border (& which border?)

time optimalization:
1. clear last item from history every 4 timesteps (we only use the current and previous state and the one before that)
2. interval of states to be interpreted: skip N frames before evaluation next state
3. Since the rewards are so sparse, maybe use Imitation learning instead of DQN --> we are "experts" since we know the tactic of the blue bot. we can use this to teach our bot how to defeat the other agent.


In [1]:
from VisualModule import AgentEnvironment
from DQN_Agent import NeurosmashAgent

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

import gym
import math
import random
from collections import namedtuple
from itertools import count
from PIL import Image
import os

from stopwatch import Stopwatch


Using TensorFlow backend.


In [2]:
model_output_dir = "output/model_output/"

if not os.path.exists(model_output_dir):
    os.makedirs(model_output_dir)

show_images = False
skip_frames = 10
state_size =  11 # agent_pos, enemy_pos, vec_agent, vec_enemy, rel_pos_enemy, done
action_size = 3
episode_count = 1000
batch_size = 32
size       = 768         # Please check the Updates section above for more details
timescale  = 10           # Please check the Updates section above for more details

In [3]:
environment = AgentEnvironment(size=size, timescale=timescale)
agent = NeurosmashAgent(state_size = state_size, action_size = action_size) # action size: move in x or y direction, or do nothing

In [4]:
def direction(agent_path, enemy_path):
    A_X = (agent_path[-1] - np.array(agent_path[-2]))[0]
    A_Y = -(agent_path[-1] - np.array(agent_path[-2]))[1]
    E_X = (enemy_path[-1] - np.array(enemy_path[-2]))[0]
    E_Y = -(enemy_path[-1] - np.array(enemy_path[-2]))[1]
    return [A_X,A_Y],[E_X,E_Y]

def do_action(action, total_steps, eval_pic):
    stopwatch = Stopwatch() 
    stopwatch.start()
    info, reward, agent_coord, enemy_coord, following_state = environment.simpleCoord(action, 0, eval_pic)
    stopwatch.stop()
    #print(f"Total time for do action: {stopwatch.duration}")
    if len(environment.agent_path) < 2:
        distance = 500 # Initial distance, only for initialisation
        agent_direction = [1,0] # By definition of facing each other
        enemy_direction = [-1,0]
    else:
        distance = np.sqrt(np.square(np.array(list(np.array(agent_coord)- np.array(enemy_coord))).sum(axis=0)))
        # Extract all variables 
        agent_direction, enemy_direction  = direction(environment.agent_path, environment.enemy_path)

    rel_pos_enemy = np.array(enemy_coord) - np.array(agent_coord)
    return info, reward, np.array(agent_coord), np.array(enemy_coord), agent_direction, enemy_direction, distance, rel_pos_enemy, following_state


In [5]:
def init_environment(env, agent_here):
    info, reward, state = env.reset() 
    agent_trajectories = []
    enemy_trajectories = []
    
    for i in range(3):
        action = agent_here.act(3) # get next action
        #pre_state_img = np.flip(np.array(state).reshape(3,256,256).transpose(1,2,0),0)
        step_number_now = i+1
        info, reward, agent_pos, enemy_pos, agent_direction, enemy_direction, distance, relative_pos_enemy, next_state = do_action(action, step_number_now, True)  

        #post_state_img = np.flip(np.array(next_state).reshape(3,256,256).transpose(1,2,0),0)

        #agent_pos, enemy_pos = env_feat.coord(pre_state_img, post_state_img)
        agent_trajectories.append(list(agent_pos))
        enemy_trajectories.append(list(enemy_pos))
        
    return info, reward, next_state, agent_trajectories, enemy_trajectories, agent_direction, relative_pos_enemy, enemy_direction

In [ ]:
for e in range(episode_count):
    status, reward, next_state, agent_trajectories, enemy_trajectories, agent_dir, relative_pos_enemy, enemy_dir = init_environment(environment, agent)
    done = False
    total_reward = 0
    total_timesteps = 4
    small_state = [agent_trajectories[-1][0], agent_trajectories[-1][1], enemy_trajectories[-1][0], enemy_trajectories[-1][1], [agent_dir[0]], [agent_dir[1]], [relative_pos_enemy[0]], [relative_pos_enemy[1]], [enemy_dir[0]], [enemy_dir[1]], done]#"agent direction", "relative position enemy", "enemy direction" ]
    distances = []
    evaluate_frame = False

    while done == False:
        if total_timesteps % skip_frames == 0:
            evaluate_frame = True
        else:
            evaluate_frame = False
            
        action = agent.act(small_state) #step(info, reward, state)
        #print(f"agent chooses action: {action}")
        stopwatch = Stopwatch() 
        stopwatch.start()
        status, reward, agent_pos, enemy_pos, agent_dir, enemy_dir, distance, enemy_pos_rel, next_state = do_action(action, total_timesteps, evaluate_frame)   
        stopwatch.stop()
        #print(f"Total time for one step: {stopwatch.duration}")
        
        total_reward += reward

        if status == 1:
            print(f"Game is finished, \n your final reward is: {total_reward}, duration was {total_timesteps} timesteps")
            done = True
        
        agent_trajectories.append(list(agent_pos))
        enemy_trajectories.append(list(enemy_pos))
        distances.append(distance)
        
        done_list = [done]
        next_small_state = [agent_trajectories[-1][0], agent_trajectories[-1][1], enemy_trajectories[1][0], enemy_trajectories[1][1], agent_dir[0], agent_dir[1], enemy_pos_rel[0], enemy_pos_rel[1], enemy_dir[0], enemy_dir[1], done]  
    
        next_small_state = np.reshape(next_small_state, [1, state_size]) # why?

        if evaluate_frame:
            agent.remember(small_state, action, reward, next_small_state, list(done_list))
        small_state = next_small_state
        total_timesteps += 1

    if len(agent.memory) > batch_size:
        agent.train(batch_size)
        print("train")

    if e % 50 == 0:
        agent.save(model_output_dir + "weights_"+ '{:04d}'.format(e) + ".hdf5")
            
    

total time for current frame (with analysis): 0.21403508699586382
total time for current frame (with analysis): 0.21938864100229694
total time for current frame (no analysis): 0.11570855299942195
total time for current frame (no analysis): 0.10375424399535405
total time for current frame (no analysis): 0.09794914600206539
total time for current frame (no analysis): 0.106223474998842
total time for current frame (no analysis): 0.14874026200413937
total time for current frame (no analysis): 0.15192896199732786
total time for current frame (with analysis): 0.21749390500190202
total time for current frame (no analysis): 0.14607496099779382
total time for current frame (no analysis): 0.14396978399599902
total time for current frame (no analysis): 0.1002740289986832
total time for current frame (no analysis): 0.10182125499704853
total time for current frame (no analysis): 0.0976423350002733
total time for current frame (no analysis): 0.09888279299775604
total time for current frame (no analy

total time for current frame (with analysis): 0.20603887100151042
total time for current frame (no analysis): 0.0996730810002191
total time for current frame (no analysis): 0.09419245200115256
total time for current frame (no analysis): 0.11519911199866328
total time for current frame (no analysis): 0.09494500299479114
total time for current frame (no analysis): 0.09756384500360582
total time for current frame (no analysis): 0.09538343500025803
total time for current frame (no analysis): 0.0958762160007609
total time for current frame (no analysis): 0.09386370299762348
total time for current frame (no analysis): 0.1375283910019789
total time for current frame (with analysis): 0.27257218600425404
total time for current frame (no analysis): 0.11571417099912651
total time for current frame (no analysis): 0.09598343299876433
total time for current frame (no analysis): 0.09719257200049469
total time for current frame (no analysis): 0.09490460799861467
total time for current frame (no analys